## Connect Proxy client

In [1]:
from tempfile import NamedTemporaryFile
import bitcoin.rpc

```bash
docker run --rm -it -p 8332:8332 --name=bitcoin ruimarinho/bitcoin-core   -printtoconsole   -regtest=1   -rpcallowip=0.0.0.0/0 -rpcport=8332 -rpcpassword=bar -rpcuser=foo
```

In [2]:
conf_str = """
rpcuser=df2EDbbsD
rpcpassword=46sDGGFD2
rpcport=8332
rpchost=0.0.0.0
rpctimeout=10000000
"""

In [3]:
class Proxy(object):
    """Simple proxy class to bitcoin.rpc.Proxy to deal with timeouts"""
    def __init__(self, conf_str):
        self.conf_file = NamedTemporaryFile(mode='w+t')
        self.conf_file.write(conf_str)
    
    def __getattr__(self, name):
        if name.startswith('_'):
            return getattr(self, name)
        self.conf_file.seek(0)
        proxy = bitcoin.rpc.Proxy(btc_conf_file=self.conf_file.name,
                                  timeout=3600)
        self._proxy = proxy # DEBUG
        return getattr(proxy, name)
    
    def __del__(self):
        self.conf_file.close()

In [4]:
proxy = Proxy(conf_str=conf_str)

In [6]:
proxy.generate(101)

<generator object Proxy.generate.<locals>.<genexpr> at 0x10566e360>

In [7]:
proxy.getinfo()

{'balance': 5000000000,
 'blocks': 101,
 'connections': 0,
 'deprecation-warning': 'WARNING: getinfo is deprecated and will be fully removed in 0.16. Projects should transition to using getblockchaininfo, getnetworkinfo, and getwalletinfo before upgrading to 0.16',
 'difficulty': Decimal('4.656542373906925E-10'),
 'errors': '',
 'keypoololdest': 1509209407,
 'keypoolsize': 1999,
 'paytxfee': 0,
 'protocolversion': 70015,
 'proxy': '',
 'relayfee': Decimal('0.00001000'),
 'testnet': False,
 'timeoffset': 0,
 'version': 150001,
 'walletversion': 139900}

## Create timelock transaction

In [8]:
import hashlib
import arrow

import bitcoin
from bitcoin.core import COutPoint, CMutableTxOut, CMutableTxIn, CMutableTransaction, COIN, b2lx, lx
from bitcoin.wallet import CBitcoinAddress, CBitcoinSecret, P2SHBitcoinAddress
from bitcoin.core.script import (
        OP_CHECKLOCKTIMEVERIFY, OP_DROP, OP_CHECKSIG,
        CScript,
        SignatureHash, SIGHASH_ALL,
)
from bitcoin.core.scripteval import VerifyScript, SCRIPT_VERIFY_P2SH

In [9]:
bitcoin.SelectParams('regtest')

In [10]:
h = hashlib.sha256(b'secret password').digest()
dst_secret = CBitcoinSecret.from_secret_bytes(h)
dst_secret

CBitcoinSecret('cNWQk3ZX4yVtK3dAqVQ52RKw9j4q43B6dMK6M172qgMbqNDLXQYh')

In [11]:
# https://bitcoin.stackexchange.com/questions/39119/how-is-time-encoded-bip65-in-scripts
curr_time = arrow.now()
lock_time = curr_time.shift(minutes=1)
lock_time = lock_time.timestamp
lock_time

1509209554

In [12]:
dst_script = CScript([lock_time, OP_CHECKLOCKTIMEVERIFY, OP_DROP, dst_secret.pub, OP_CHECKSIG])
dst_script

CScript([x('d2b5f459'), OP_CHECKLOCKTIMEVERIFY, OP_DROP, x('032a96f87371e3f6e721606ab8362f6cb8fae97d611ad9ce693e9127c2c8bf33bb'), OP_CHECKSIG])

In [13]:
dst_addr = P2SHBitcoinAddress.from_redeemScript(dst_script)
dst_addr

P2SHBitcoinAddress('2N3LTuVKsty7QAX6Y4RAnPmiH86Z2jSjtdE')

In [14]:
utxos = proxy.listunspent()
utxos

[{'address': P2PKHBitcoinAddress('n2w4KybXLcWY9JciipTKfxAdiJj4nfec9f'),
  'amount': 5000000000,
  'confirmations': 101,
  'outpoint': COutPoint(lx('f06f256ba880892e5f4f57807de975b3eb426423a0111282e1ec6f49ca716002'), 0),
  'safe': True,
  'scriptPubKey': CScript([x('03844e98cde1f0d9623b05b05452586a6c47daa9b19f815fc74a1e23c6ddf6c08a'), OP_CHECKSIG]),
  'solvable': True,
  'spendable': True}]

In [15]:
utxo = utxos[0]
utxo

{'address': P2PKHBitcoinAddress('n2w4KybXLcWY9JciipTKfxAdiJj4nfec9f'),
 'amount': 5000000000,
 'confirmations': 101,
 'outpoint': COutPoint(lx('f06f256ba880892e5f4f57807de975b3eb426423a0111282e1ec6f49ca716002'), 0),
 'safe': True,
 'scriptPubKey': CScript([x('03844e98cde1f0d9623b05b05452586a6c47daa9b19f815fc74a1e23c6ddf6c08a'), OP_CHECKSIG]),
 'solvable': True,
 'spendable': True}

In [16]:
cltv_txid = proxy.sendtoaddress(dst_addr, 10 * COIN)
cltv_txid

b'\xd2\x01%\xd9\x99\xb85\xcfk\xc2\xa5\xad\x97\xb7\xdc\xdc&\xd9:B\xf5\x96\xcd{\xd9 68}r!\xd8'

In [17]:
cltv_tx = proxy.getrawtransaction(cltv_txid)
cltv_tx

CTransaction((CTxIn(COutPoint(lx('f06f256ba880892e5f4f57807de975b3eb426423a0111282e1ec6f49ca716002'), 0), CScript([x('3045022100b0cb511fbd2947f0fa404d373239a940e40a9b1fcac3cdb08fbdc7deb02b8ab2022027261b5b7a7600cc236a2f3fd070f72dfe0e0bb25323f4a27150dca24f548b5201')]), 0xfffffffe),), (CTxOut(10.0*COIN, CScript([OP_HASH160, x('6eaef48b456d2e84cb6e7653cc7d352b33aada8e'), OP_EQUAL])), CTxOut(39.999962*COIN, CScript([OP_DUP, OP_HASH160, x('abbed70287753c535626ffef1325ce0d411a43e7'), OP_EQUALVERIFY, OP_CHECKSIG]))), 101, 2)

In [18]:
proxy.generate(1)

<generator object Proxy.generate.<locals>.<genexpr> at 0x105732ca8>

In [19]:
proxy.listunspent()

[{'address': P2PKHBitcoinAddress('n2w4KybXLcWY9JciipTKfxAdiJj4nfec9f'),
  'amount': 5000000000,
  'confirmations': 101,
  'outpoint': COutPoint(lx('0943ceb342dae65b6a727b775253a2b2904976c7eebe807dde8ec903d08d95ae'), 0),
  'safe': True,
  'scriptPubKey': CScript([x('03844e98cde1f0d9623b05b05452586a6c47daa9b19f815fc74a1e23c6ddf6c08a'), OP_CHECKSIG]),
  'solvable': True,
  'spendable': True},
 {'address': P2PKHBitcoinAddress('mwB4TeLNpY7n3n2eD75TQvZCSYdJhoQBXW'),
  'amount': 3999996200,
  'confirmations': 1,
  'outpoint': COutPoint(lx('d821727d383620d97bcd96f5423ad926dcdcb797ada5c26bcf35b899d92501d2'), 1),
  'safe': True,
  'scriptPubKey': CScript([OP_DUP, OP_HASH160, x('abbed70287753c535626ffef1325ce0d411a43e7'), OP_EQUALVERIFY, OP_CHECKSIG]),
  'solvable': True,
  'spendable': True}]

In [32]:
outpoint = COutPoint(cltv_txid, 0)
txin = CMutableTxIn(outpoint, nSequence=1)
txin

CTxIn(COutPoint(lx('d821727d383620d97bcd96f5423ad926dcdcb797ada5c26bcf35b899d92501d2'), 0), CScript([]), 0x1)

In [33]:
dst_addr2 = proxy.getnewaddress()
dst_addr2

P2PKHBitcoinAddress('mwBLxhujHjW8TAtsAJKskgFemU3ZtdWgB7')

In [34]:
txout = CMutableTxOut(9.99*COIN,
                      dst_addr2.to_scriptPubKey())
txout

CTxOut(9.99*COIN, CScript([OP_DUP, OP_HASH160, x('abcc9d412e79ea40b0b261bec70596cfc158ea56'), OP_EQUALVERIFY, OP_CHECKSIG]))

In [35]:
curr_time = arrow.now()
curr_time = curr_time.timestamp
curr_time

1509209683

In [36]:
spend_tx = CMutableTransaction([txin], [txout], nLockTime=curr_time)
spend_tx

CTransaction([CTxIn(COutPoint(lx('d821727d383620d97bcd96f5423ad926dcdcb797ada5c26bcf35b899d92501d2'), 0), CScript([]), 0x1)], [CTxOut(9.99*COIN, CScript([OP_DUP, OP_HASH160, x('abcc9d412e79ea40b0b261bec70596cfc158ea56'), OP_EQUALVERIFY, OP_CHECKSIG]))], 1509209683, 1)

In [37]:
sighash = SignatureHash(dst_script, spend_tx, 0, SIGHASH_ALL)
sighash

b'\xf8\xca\x01\t\x9f\xdd\xf2\x89[\x96*\xcd8)\x11\xad63\x1a\xf7\xc5\x13\x93\x93\x10\xbd\x1b\x10\x97S\x0c\xf2'

In [38]:
sig = dst_secret.sign(sighash) + bytes([SIGHASH_ALL])
sig

b'0E\x02!\x00\xa3Q\xb9\x1fL\x8e\x15VL.d\x1e\xe3$\x08\xba\x87\xf3\xbd\xd6@\x04\xf6Gz\xf0\xe8\xcd*\x8d<W\x02 v\x8fz\xfd\x16)\x80\xbfA(B\x8f\xce+J\xa1Ey\xd4\xa1\xe2\xa0\xc0`\xa2\xeb=\x9d\x0b\xa7\x857\x01'

In [39]:
sig_script = CScript([sig, dst_script])
sig_script

CScript([x('3045022100a351b91f4c8e15564c2e641ee32408ba87f3bdd64004f6477af0e8cd2a8d3c570220768f7afd162980bf4128428fce2b4aa14579d4a1e2a0c060a2eb3d9d0ba7853701'), x('04d2b5f459b17521032a96f87371e3f6e721606ab8362f6cb8fae97d611ad9ce693e9127c2c8bf33bbac')])

In [40]:
txin.scriptSig = sig_script

In [41]:
spend_tx

CTransaction([CTxIn(COutPoint(lx('d821727d383620d97bcd96f5423ad926dcdcb797ada5c26bcf35b899d92501d2'), 0), CScript([x('3045022100a351b91f4c8e15564c2e641ee32408ba87f3bdd64004f6477af0e8cd2a8d3c570220768f7afd162980bf4128428fce2b4aa14579d4a1e2a0c060a2eb3d9d0ba7853701'), x('04d2b5f459b17521032a96f87371e3f6e721606ab8362f6cb8fae97d611ad9ce693e9127c2c8bf33bbac')]), 0x1)], [CTxOut(9.99*COIN, CScript([OP_DUP, OP_HASH160, x('abcc9d412e79ea40b0b261bec70596cfc158ea56'), OP_EQUALVERIFY, OP_CHECKSIG]))], 1509209683, 1)

In [43]:
proxy.generate(100)

<generator object Proxy.generate.<locals>.<genexpr> at 0x1057e04c0>

In [44]:
txid2 = proxy.sendrawtransaction(spend_tx)
b2lx(txid2)

'a5147def4dafee266145fbf8b82c152bf8296fe389fce974520ca866df03bb8b'

In [45]:
proxy.generate(1)

<generator object Proxy.generate.<locals>.<genexpr> at 0x1057e0b48>

In [46]:
proxy.gettransaction(txid2)

{'amount': Decimal('9.99000000'),
 'bip125-replaceable': 'no',
 'blockhash': '79d92028600ebfd5829d04aa621eb85c5eff704abce901b91cf7bd0ccce069a1',
 'blockindex': 1,
 'blocktime': 1509209729,
 'confirmations': 1,
 'details': [{'account': '',
   'address': 'mwBLxhujHjW8TAtsAJKskgFemU3ZtdWgB7',
   'amount': Decimal('9.99000000'),
   'category': 'receive',
   'label': '',
   'vout': 0}],
 'hex': '0100000001d20125d999b835cf6bc2a5ad97b7dcdc26d93a42f596cd7bd92036387d7221d80000000074483045022100a351b91f4c8e15564c2e641ee32408ba87f3bdd64004f6477af0e8cd2a8d3c570220768f7afd162980bf4128428fce2b4aa14579d4a1e2a0c060a2eb3d9d0ba78537012a04d2b5f459b17521032a96f87371e3f6e721606ab8362f6cb8fae97d611ad9ce693e9127c2c8bf33bbac0100000001c0878b3b000000001976a914abcc9d412e79ea40b0b261bec70596cfc158ea5688ac53b6f459',
 'time': 1509209713,
 'timereceived': 1509209713,
 'txid': 'a5147def4dafee266145fbf8b82c152bf8296fe389fce974520ca866df03bb8b',
 'walletconflicts': []}

In [14]:
src_addr = utxo['address']
src_addr

P2PKHBitcoinAddress('n3iUoD7zqEXAwH6RCwnRpxQ2RVvGH1xSaQ')

In [11]:
src_secret = proxy.dumpprivkey(src_addr)
src_secret

CBitcoinSecret('cN9RYdbvGyjsq6xjRLi28tJC6Dco9J5YkByK3o5xVwqS81eZrfFZ')

1509205855

In [15]:
src_addr

P2PKHBitcoinAddress('n3iUoD7zqEXAwH6RCwnRpxQ2RVvGH1xSaQ')

In [16]:
src_secret.pub

CPubKey(b'\x02\xa1?!\xb2\xec\xc5&\xde\x11Z|eb\xc2\x90h\x0b\x06x\x83U4\x9d\x96\xc3\x96\xad\xef\xb9\x8f\x95\xc4')

NameError: name 'privkey' is not defined

In [ ]:
addr = P2SHBitcoinAddress.from_redeemScript(redeemScript)

In [14]:
utxo = proxy.listunspent()[0]
utxo

{'account': '',
 'address': P2PKHBitcoinAddress('mjjy43wHLMeoGy78Ju8DF9hBWZM1qyRss6'),
 'amount': 999900000,
 'confirmations': 0,
 'outpoint': COutPoint(lx('497405713e6c094663245d28f465045e982cfece36d9b5a24bbcd9c68bad7952'), 0),
 'safe': False,
 'scriptPubKey': CScript([OP_DUP, OP_HASH160, x('2e568b3aca5284dabb37107d2fdcefbb8aad851e'), OP_EQUALVERIFY, OP_CHECKSIG]),
 'solvable': True,
 'spendable': True}

In [15]:
txin = CMutableTxIn(utxo['outpoint'], nSequence=1)
txin

CTxIn(COutPoint(lx('497405713e6c094663245d28f465045e982cfece36d9b5a24bbcd9c68bad7952'), 0), CScript([]), 0x1)

In [17]:
src_addr = utxo['address']
src_addr

P2PKHBitcoinAddress('mjjy43wHLMeoGy78Ju8DF9hBWZM1qyRss6')

In [22]:
dest_addr = src_addr

In [45]:
txout = CMutableTxOut(9.9*COIN,
                      #dest_addr.to_scriptPubKey())
                      script)

In [46]:
tx = CMutableTransaction([txin], [txout])
tx

CTransaction([CTxIn(COutPoint(lx('497405713e6c094663245d28f465045e982cfece36d9b5a24bbcd9c68bad7952'), 0), CScript([x('304402203d4afe4c534e84f63912fe6af1b51daf0d7424f22070ac903dcb61d25ae8e5570220325d0bd51914831e21ef0c7136a87f5a96de1505d903814b1c42abacb518447601'), x('02ff97629efcb01518a6a5a41a40c6b2fbb93e9c726ebfe84b2b022189b71e4a2c')]), 0x1)], [CTxOut(9.9*COIN, CScript([x('6ac0ef59'), OP_CHECKLOCKTIMEVERIFY, OP_DROP, x('02b568858a407a8721923b89df9963d30013639ac690cce5f555529b77b83cbfc7'), OP_CHECKSIG]))], 0, 1)

In [39]:
src_secret = proxy.dumpprivkey(src_addr)
src_secret

CBitcoinSecret('cR6Nh29YCohUkf1GQ4WjX1uxRwUoESaCRBb7QdePLCWvhPzJLEYc')

In [47]:
sighash = SignatureHash(src_addr.to_scriptPubKey(), tx, 0, SIGHASH_ALL)
sighash

b"A\x15-\xe0(\xecO\xce\xe8R'\x87\xc0\xd0P\xb1\xa1\xdf\x90\n\xceO~\x15\xb0\xba\x99\xa8%:9\xb2"

In [48]:
sig = src_secret.sign(sighash) + bytes([SIGHASH_ALL])
sig

b"0D\x02 ]\x0e\r\xaf\r73\xc9\xfax}\xbe$U\xa6\xb9'@\x8dY\xff\xc3\x9e\xfee\xbbg\x1a\x1c\xe9;\x15\x02 X\x96\xd1\x9a\xc9\xb1T\xfd8\xa1g\xf0nU\xd42\xafj\xa8\x0c10\xca@\xf3,\xc7Y\x93\xdf?\x14\x01"

In [49]:
txin.scriptSig = CScript([sig, src_secret.pub])
txin.scriptSig

CScript([x('304402205d0e0daf0d3733c9fa787dbe2455a6b927408d59ffc39efe65bb671a1ce93b1502205896d19ac9b154fd38a167f06e55d432af6aa80c3130ca40f32cc75993df3f1401'), x('02ff97629efcb01518a6a5a41a40c6b2fbb93e9c726ebfe84b2b022189b71e4a2c')])

In [50]:
txid = proxy.sendrawtransaction(tx)
txid = b2lx(txid)
txid

'16ae89fec61792e9015bfb3fced5f296e4c3b71c36e1cbe00266ffd4e9a5c46e'

## Spend timelock

In [180]:
proxy = bitcoin.rpc.Proxy(btc_conf_file=conf_file.name, timeout=3600)

In [63]:
utxo2 = proxy.getrawtransaction(lx(txid))
utxo2

CTransaction((CTxIn(COutPoint(lx('497405713e6c094663245d28f465045e982cfece36d9b5a24bbcd9c68bad7952'), 0), CScript([x('304402205d0e0daf0d3733c9fa787dbe2455a6b927408d59ffc39efe65bb671a1ce93b1502205896d19ac9b154fd38a167f06e55d432af6aa80c3130ca40f32cc75993df3f1401'), x('02ff97629efcb01518a6a5a41a40c6b2fbb93e9c726ebfe84b2b022189b71e4a2c')]), 0x1),), (CTxOut(9.9*COIN, CScript([x('6ac0ef59'), OP_CHECKLOCKTIMEVERIFY, OP_DROP, x('02b568858a407a8721923b89df9963d30013639ac690cce5f555529b77b83cbfc7'), OP_CHECKSIG])),), 0, 1)

In [74]:
outpoint = COutPoint(utxo2.GetTxid(), 0)
outpoint

COutPoint(lx('16ae89fec61792e9015bfb3fced5f296e4c3b71c36e1cbe00266ffd4e9a5c46e'), 0)

In [75]:
txin2 = CMutableTxIn(outpoint, nSequence=1)
txin2

CTxIn(COutPoint(lx('16ae89fec61792e9015bfb3fced5f296e4c3b71c36e1cbe00266ffd4e9a5c46e'), 0), CScript([]), 0x1)

In [79]:
prevout = utxo2.vout[0]
prevout

CTxOut(9.9*COIN, CScript([x('6ac0ef59'), OP_CHECKLOCKTIMEVERIFY, OP_DROP, x('02b568858a407a8721923b89df9963d30013639ac690cce5f555529b77b83cbfc7'), OP_CHECKSIG]))

In [81]:
dst_addr2 = proxy.getnewaddress()
dst_addr2

P2PKHBitcoinAddress('myVRap1agJZmbhQvaapPYDBLnu66jbwh4j')

In [82]:
txout2 = CMutableTxOut(9.89*COIN,
                       dst_addr2.to_scriptPubKey())
txout2

CTxOut(9.89*COIN, CScript([OP_DUP, OP_HASH160, x('c528363a4b1210081812c22f7f1df0de9321d3db'), OP_EQUALVERIFY, OP_CHECKSIG]))

In [172]:
nLockTime = arrow.now().timestamp
nLockTime

1509024919

In [173]:
tx2 = CMutableTransaction([txin2], [txout2], nLockTime=nLockTime)
tx2

CTransaction([CTxIn(COutPoint(lx('16ae89fec61792e9015bfb3fced5f296e4c3b71c36e1cbe00266ffd4e9a5c46e'), 0), CScript([x('3045022100f2343220b82c6ec3579900ebfeadca45da7cc82d4d58faf5ddffcee4526d2866022048a6673001772c1887192b2b4a511fae78c7dbc841eacc0000e330e42aca31cd01')]), 0x1)], [CTxOut(9.89*COIN, CScript([OP_DUP, OP_HASH160, x('c528363a4b1210081812c22f7f1df0de9321d3db'), OP_EQUALVERIFY, OP_CHECKSIG]))], 1509024919, 1)

In [87]:
dst_secret2 = proxy.dumpprivkey(dst_addr2)
dst_secret2

CBitcoinSecret('cTt9MVXfyBcZKFHWzFZz68d1U1C184JDYHSyhUyHTyVYF65ktPF5')

In [174]:
prevout.scriptPubKey

CScript([x('6ac0ef59'), OP_CHECKLOCKTIMEVERIFY, OP_DROP, x('02b568858a407a8721923b89df9963d30013639ac690cce5f555529b77b83cbfc7'), OP_CHECKSIG])

In [175]:
sighash2 = SignatureHash(prevout.scriptPubKey, tx2, 0, SIGHASH_ALL)
sighash2

b'\x8d\xce7|#\xac\xe1\xf0\xef\x96R\x15\xd4o\xd9\xa1jK\x90\x07\xc3=\x817^\xbd\x86\xe6\xd7\xd9\xcdQ'

In [176]:
sig2 = privkey.sign(sighash2) + bytes([SIGHASH_ALL])
sig2

b'0D\x02 !\xfbDR\xd8y\xd4"W\xb5\xf1\x802\xe3CA\xc4q\x1e\t\xab\xe6\xf1\x84:\xf6yt\xf9\xeb\xd0\xb6\x02 P\xc3\x81x\xb26vb\x85-{\x0b\xc9q\xf8\x85\xed\xbc\x15\xe6_4"\xca^3V\x8f\xda\x8e\xc8%\x01'

In [177]:
txin2.scriptSig = CScript([sig2])
txin2.scriptSig

CScript([x('3044022021fb4452d879d42257b5f18032e34341c4711e09abe6f1843af67974f9ebd0b6022050c38178b2367662852d7b0bc971f885edbc15e65f3422ca5e33568fda8ec82501')])

In [178]:
tx2

CTransaction([CTxIn(COutPoint(lx('16ae89fec61792e9015bfb3fced5f296e4c3b71c36e1cbe00266ffd4e9a5c46e'), 0), CScript([x('3044022021fb4452d879d42257b5f18032e34341c4711e09abe6f1843af67974f9ebd0b6022050c38178b2367662852d7b0bc971f885edbc15e65f3422ca5e33568fda8ec82501')]), 0x1)], [CTxOut(9.89*COIN, CScript([OP_DUP, OP_HASH160, x('c528363a4b1210081812c22f7f1df0de9321d3db'), OP_EQUALVERIFY, OP_CHECKSIG]))], 1509024919, 1)

In [179]:
VerifyScript(txin2.scriptSig, prevout.scriptPubKey, tx2, 0, (SCRIPT_VERIFY_P2SH,))

In [182]:
txid = proxy.sendrawtransaction(tx2)
txid = b2lx(txid)
txid

'cde7b7447203f2734597990b2db9a22a06c2ddf47250472898c0f9c47fbf0e17'